In [1]:
"""
pip install opencv-contrib-python==3.4.2.16
"""
import base64
import json
import pprint
import cv2
import numpy as np
from finalProject.classes.yolo import Yolo
from finalProject.utils.drawing.draw import drawTargetFinal
from finalProject.utils.images.imagesUtils import Image
from finalProject.utils.keyPoints.AlgoritamKeyPoints import createDescriptorTarget, SiftDetectKeyPoints
from finalProject.utils.matchers.Matchers import compare_between_two_description, flannmatcher
from finalProject.utils.preprocessing.preprocess import readFromInputVideoFrames, framesExists, reduceNoise, \
    removeRemovalColor
from finalProject.utils.tracking.TrackingByYolo import source_detection_by_yolo, tracking_by_yolo
import matplotlib.pyplot as plt
import random
import pandas as pd


def sobel(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    sobel_intensity = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx, 2.0),
                                               1.0, cv2.pow(sobely, 2.0), 1.0, 0.0))
    return sobel_intensity


def sobel_keypoints(image):
    sobelImage = sobel(image)
    # norm
    image8bit = cv2.normalize(sobelImage, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    k, d = SiftDetectKeyPoints(image8bit)
    return k, d, image8bit


def forward(frames):
    outputs = []
    for frame in frames:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = sobel(frame)
        key_des_image = sobel_keypoints(frame)
        outputs.append(key_des_image)

    return outputs

def encodeToBase64(output):
    
    output[0] = cv2.resize(output[0],(40,40))
    output[1] = cv2.resize(output[1],(40,40))
    output[2] = cv2.resize(output[2],(40,40))

    output[0] = base64.b64encode(output[0]).decode('utf-8')
    output[1] = base64.b64encode(output[1]).decode('utf-8')
    output[2] = base64.b64encode(output[2]).decode('utf-8')
    
    return output



def exportToExcel(outputs):
    outputs = list(map(encodeToBase64,outputs))
    df2 = pd.DataFrame(np.array(outputs),
                   columns=['outputImage', 'img1','img2','len(k1)','len(k2)','len(match)'])
    ## need to convert image to base 64 
    
    df2.to_excel("./reports/report1.xls")
    

def cross_correction(key_des_image_source, key_des_image_target, threshold=0.8):
    (k1, d1, img1) = key_des_image_source
    (k2, d2, img2) = key_des_image_target
    match = flannmatcher(d1, d2, threshold=threshold)
    if len(match) == 0:
        return None
    output = cv2.drawMatchesKnn(img1, k1, img2, k2, match, outImg=None)
    
    # return output
    return [output,img1,img2,len(k1),len(k2),len(match)] # item = (outputs,img1,len(k1),img2,len(k2)..)


def plotSquare(images, _titles,threshold, each_column=10):
    number_images = len(images)
    rows = (number_images // each_column) 
    index = 1

    print(rows)
    fig, axes = plt.subplots(rows, each_column, figsize=(48, 24),dpi=300)
    fig.subplots_adjust(hspace=0.3, wspace=0.05)
    
    for ax,image,title in zip(axes.flat,images,_titles):
        ax.imshow(image)
        ax.set_title(title)
        index += 1


    rand = random.uniform(0, 1)
    pathX = "./figures/fig_test"+str(rand)+"_thresh_"+str(threshold)+".svg"
    print("saving on",pathX)
    plt.savefig(pathX,dpi=fig.dpi)
    plt.clf()
    plt.close()





In [2]:
# init yolo
yolo = Yolo()
yolo.initYolo()
pp = pprint.PrettyPrinter(indent=4)


[INFO] loading YOLO from disk...


In [3]:
def run(threshold=0.8):
  
    with open('./config.txt') as file_json:
        config = json.load(file_json)

        # source
        frameSource = readFromInputVideoFrames(config["source"])
        if not framesExists(frameSource):
            print("problem with source video input")
            exit(0)

        mySource = source_detection_by_yolo(frameSource, yolo,
                                            isVideo=config["source"]["isVideo"],
                                            config=config["source"])
        if mySource is None:
            print("fail to detect human on source video")
            exit(0)

        # target
        frameTarget = readFromInputVideoFrames(config["target"])
        if not framesExists(frameTarget):
            print("problem with target video input")
            exit(0)

        if not framesExists(frameTarget):
            print("problem with target video input -reduce noise")
            exit(0)

        myTargets = tracking_by_yolo(frameTarget, yolo, isVideo=config["target"]["isVideo"],
                                     config=config["target"])

        cv2.destroyAllWindows()
        # source

        sources_output = forward(mySource.frames)  # array of outputs (key,des,image)
        targets_outputs = []  # array of targets , each target have array of frames
        for target in myTargets:
            target_output = forward(target.frames)  # array of frames
            targets_outputs.append(target_output)

        outputs = []  # each frame

        for source_frame in sources_output:
            for target in targets_outputs:
                for target_frame in target:                                  ## threshold for mactch ! bigger - more lines
                    cross = cross_correction(source_frame, target_frame,threshold)
                    if cross is not None:
                        outputs.append(cross)

        exportToExcel(outputs)
                        
#         titles = ["test"] * (len(plots) - 1)
#         plotSquare(plots, titles,threshold)
#         print("finish")

In [4]:
for thresh in [0.8]:
    run(threshold=thresh)


In [11]:
s = 'AQEBAAIBAwICAwABAwIBAgIAAgcCCAQGDQELCQQFAgICAQIBAwMDBAECAAEBAgQBAwECAQACRDFFJBEYMFEFBAoGBwILBgIBAQICAgQEAAUBAgEBAAEDAQABAAICJyx+HxcHDyYkUgMGFwQCCQQDCgEBAgIBAgMDAgQAAQEBCgEBBAYDASRHKnFJOzomFj4CAgUEAgICBAMCAAMBAQIDAQMDAQEBAAMAAAENAg9OVAomT0xIFRkQFwQBAgMBAwEKBAACAQAAAgICAQIBAQEBAC06BAAHLh8pAUsrFhILFQQBAwICBgIBAwEBAwIBAQYFAAABAAEAAQIdChMLHQ9iHxsVGw4NDApKTAoVAg0EAwQFAwIFAQEAAQICAgEBAQABBC0SJAMHTQsJF0lIHSMCAQUEMR8CDwMDAAEBAwEAAAEACgEBAQIABBEfKgoGAwcTDQQGAxomAAIDPxkBAwIDAAEAAQMCAQICFVd5QX9GFgMFAAEKBwMEEhAeR0EMAwQAAAAFWFkGAgICAQABAQECAREFAQAGFAoOBAEBAAgNAAEEBQMEAQICBAIBAgECFg4BAgEDAAECAQEHERIDEA0CAgECAwMDAQMCAQEEAgIMBgUAAQEBAgY3FgIAAwABAQAABAcGCQMNBwkBAQUCAgION8S1qpNWRA8DAQECAAABAygUAgICAwEAAR0KCwYDBAQDBgKJIS04D00yGS0XVx5IFDhVcQABAQEDBQgCAQAAAAEcDAMTCAIHAgAGYy4XHyRBQ0s0Qi85NxcDFDYGAgIAAAcSEgIBAQIBHQIIChEAAQEBAw80HV5+qnlXQVs6SQ1MHhEdDQEBAQEBEQ8BAwIBAgsIFgwMAgUFBQkBAgIDAgBYET0/PCUBAQAEAgEBAAMAAgAaMgABAAJ0AxYOBgECAQEBAAMEBAMDHjMmJydJAgICAgQEAQIAAQEBAxgcAAADfhIEBAYCBgMDAgMBSB0cGx8aGAcRCQ0jJR8OGx8FAQIBAAERDwIAAV4TBwgKAQECAwECBAMFBxYHCBEQChIMAgIGAg8mQgICAAABARoZAgJ/EBECBgoIBQUEBgIBAQEDBgcGBAQFAgMBAgEAHDYtAwEBAAACChAAhQMIDgUKDwQHAgAFAgACAgEAAgEBAQMBAQACATEIJhUBAQABABsnBEUeEwIDCQsEAwEDAwIDAQMAAQECAgIBAAABAQEIMxsbFAECAQEFCgIWbwwPGBkMBDWSFgEBBgMEBQMDAwEBAgECAQIBAhYVAiEUBAIEAgkLCo6FAgADRUUDJ2YpBQIDAQIBAgIBAgEBAgQGAQIhEwIFCBQFAwsVAApfYgICBggVCIdYbhICAwIACAMEBwMHAgABAggVDAwFAAMPVGoxBQABVjoDARhEChVbUysXAgEDAgMBAAICAgECAQEAAxQIBAEBNg0QCCIZAox4AwEDAzMdOEorCAABBgEBAgIBAQEBAgECAwIKBQQAAxEvGy4MCgBwWQ0GCxcQCRcGCQYCAQQICAACAQAAAQEAAQQACAEDAwIFESQaDxgALjwoAgADHygBAQEAAgIABAQCBAMAAQEAAAEBAQwDBA4AAhoqKRYOAQ91VgEDG0saAQEBAAEAAAEEDgEBAQEBAAECAQEPAQQHAQEDAAAAAQIFamwEBxgdDwUAAwQBAAEAAAcLAgABAAABBgIACAECAgIBAQIAAQIBBFMnAwUOFREBAgIBAQAAAAIIBgMGAAABAgECAAMBAAECAwICAQABAQRmcgcBAQ0TAQABAwABAQEEAgUIAwIAAQEBAQUCAAEBCAEBAAABAAICdVQMAQAiBgEBAQIBAQABAQEBAwEDAQAAAAIBAQABAgUAAAAAAAEBAD0wIAEDKRAAAAAAAQIAAgEBAQcDAgEAAAABAwEBAQIGAAEBAAABAQEXOD4CAyYRAQIDAAABAAIBAQECBQIBAQEAAQUAAQADAwQBAQEDAhkLBGxmDgsIEAMAAgEBBQEBAQEAABADAQABAwEBAwICAwMCAAIAAQFAGywSPwEACCcLAgIABgUAAAABAAADCwABAQIBAQEAAAICAgECAgEBCAcPAj8FAQELDQAAAgMCAgEAAQIAAQcBAQECAQEEAgEDAQoCAgIDAA=='

In [12]:
r = base64.b64decode(s)

In [13]:
q = np.frombuffer(r, dtype=np.float64)

In [16]:
plt.show()